In [1]:
import pandas as pd

In [36]:
df = pd.read_csv('https://raw.githubusercontent.com/lavibula/SentimentAnalysis-with-Vietnamese-reviews/main/Data_preprocessed/Train.csv')
df = df.dropna().reset_index()
df = df.drop(columns = ['Unnamed: 0', 'index', 'sentence'])
df

,sentiment,topic,sentence_dropped
0,2,1,slide giáo_trình đầy_đủ
1,2,0,nhiệt_tình giảng_dạy gần_gũi sinh_viên
2,0,1,đi học đầy_đủ chuyên_cần
3,0,0,NOT áp_dụng công_nghệ thông_tin thiết_bị giảng...
4,2,0,thầy giảng hay nhiều bài_tập ví_dụ lớp
...,...,...,...
11380,0,1,môn game học hai NOT NOT hài_lòng vô_cùng NOT ...
11381,2,0,cảm_ơn cô nhiều
11382,0,0,giao bài_tập quá nhiều
11383,2,0,giáo_viên dạy hiểu nhiệt_tình


In [37]:
df['sentence_dropped'] = df['sentence_dropped'].apply(lambda x: x.lower())

In [38]:
df = df[['sentence_dropped'] + [col for col in df if col != 'sentence_dropped']]
df

,sentence_dropped,sentiment,topic
0,slide giáo_trình đầy_đủ,2,1
1,nhiệt_tình giảng_dạy gần_gũi sinh_viên,2,0
2,đi học đầy_đủ chuyên_cần,0,1
3,not áp_dụng công_nghệ thông_tin thiết_bị giảng...,0,0
4,thầy giảng hay nhiều bài_tập ví_dụ lớp,2,0
...,...,...,...
11380,môn game học hai not not hài_lòng vô_cùng not ...,0,1
11381,cảm_ơn cô nhiều,2,0
11382,giao bài_tập quá nhiều,0,0
11383,giáo_viên dạy hiểu nhiệt_tình,2,0


In [3]:
df.isna().sum()

Unnamed: 0              0
sentence            67740
sentiment               0
topic                   0
sentence_dropped      314
dtype: int64

In [39]:
# Import các thư viện cần thiết
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

### RF

In [41]:
# Tách features (câu đánh giá) và labels (cảm xúc)
X = df['sentence_dropped']
y = df['sentiment']

# Chuyển đổi dữ liệu text sang vector đặc trưng bằng CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Xây dựng mô hình Random Forest
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

# Đánh giá mô hình trên tập kiểm tra
y_pred = rf_classifier.predict(X_test)
print(classification_report(y_test, y_pred))

# Lưu ý: Bạn cũng có thể tinh chỉnh siêu tham số của mô hình RF bằng cách sử dụng các kỹ thuật tinh chỉnh siêu tham số như GridSearchCV hoặc RandomizedSearchCV.

              precision    recall  f1-score   support

           0       0.85      0.86      0.85      1049
           1       0.32      0.23      0.27        77
           2       0.87      0.88      0.87      1151

    accuracy                           0.85      2277
   macro avg       0.68      0.66      0.67      2277
weighted avg       0.84      0.85      0.84      2277



In [44]:
X

<11385x1713 sparse matrix of type '<class 'numpy.int64'>'
	with 59119 stored elements in Compressed Sparse Row format>

In [45]:
# Tách features (câu đánh giá) và labels (cảm xúc)
X = df['sentence_dropped']
y = df['topic']

# Chuyển đổi dữ liệu text sang vector đặc trưng bằng CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Xây dựng mô hình Random Forest
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

# Đánh giá mô hình trên tập kiểm tra
y_pred = rf_classifier.predict(X_test)
print(classification_report(y_test, y_pred))

# Lưu ý: Bạn cũng có thể tinh chỉnh siêu tham số của mô hình RF bằng cách sử dụng các kỹ thuật tinh chỉnh siêu tham số như GridSearchCV hoặc RandomizedSearchCV.

              precision    recall  f1-score   support

           0       0.90      0.92      0.91      1637
           1       0.71      0.64      0.67       448
           2       0.88      0.87      0.88        93
           3       0.36      0.41      0.38        99

    accuracy                           0.84      2277
   macro avg       0.71      0.71      0.71      2277
weighted avg       0.84      0.84      0.84      2277



In [46]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [48]:
import torch
from transformers import AutoModel, AutoTokenizer

phobert = AutoModel.from_pretrained("vinai/phobert-base-v2")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

# INPUT TEXT MUST BE ALREADY WORD-SEGMENTED!
sentence = 'Chúng_tôi là những nghiên_cứu_viên .'  

input_ids = torch.tensor([tokenizer.encode(sentence)])

with torch.no_grad():
    features = phobert(input_ids)  # Models outputs are now tuples

## With TensorFlow 2.0+:
# from transformers import TFAutoModel
# phobert = TFAutoModel.from_pretrained("vinai/phobert-base")

config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

In [50]:
# INPUT TEXT MUST BE ALREADY WORD-SEGMENTED!
sentence = 'Chúng_tôi là những nghiên_cứu_viên .'  

input_ids = torch.tensor([tokenizer.encode(sentence)])

with torch.no_grad():
    features = phobert(input_ids)  # Models outputs are now tuples

In [52]:
features[0][0]

tensor([[ 3.8362e-02,  7.0703e-01, -1.3202e-01,  ..., -9.7446e-02,
          2.5193e-01,  3.4828e-01],
        [ 2.1041e-01,  2.3984e-01,  9.1061e-03,  ..., -3.2422e-04,
         -1.7492e-01,  4.0127e-02],
        [ 2.3745e-01,  9.8414e-03, -1.6509e-01,  ..., -4.3379e-02,
         -7.5782e-02,  4.6838e-02],
        ...,
        [ 2.3041e-01,  3.7583e-01,  1.7602e-02,  ...,  7.8472e-02,
          1.8661e-01,  5.2052e-02],
        [-2.3020e-01,  5.0276e-01,  1.0913e-01,  ..., -7.3260e-02,
          1.4339e-01,  1.8320e-01],
        [ 1.8813e-01,  6.2870e-01, -2.4809e-01,  ..., -4.8115e-02,
          1.6404e-01,  4.7204e-01]])

In [56]:
# features.last_hidden_state[:, 0, :]

In [60]:
X_train

<9108x1713 sparse matrix of type '<class 'numpy.int64'>'
	with 47345 stored elements in Compressed Sparse Row format>

In [76]:
# Tách features (câu đánh giá) và labels (cảm xúc)
X = df['sentence_dropped']
y = df['topic']

# # Chuyển đổi dữ liệu text sang vector đặc trưng bằng CountVectorizer
# vectorizer = CountVectorizer()
# X = vectorizer.fit_transform(X)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tính toán embeddings cho một câu đánh giá
def compute_embeddings(sentence):
    # Tokenize câu đánh giá và thêm token đặc biệt
    # INPUT TEXT MUST BE ALREADY WORD-SEGMENTED!
#     sentence = 'Chúng_tôi là những nghiên_cứu_viên .'  

    input_ids = torch.tensor([tokenizer.encode(sentence)])

    with torch.no_grad():
        features = phobert(input_ids)  # Models outputs are now tuples
    
    # Lấy embeddings từ lớp cuối cùng (CLS token)
    cls_embedding = features.last_hidden_state[:, 0, :]
    
    return cls_embedding

# # Tính toán embeddings cho tất cả các câu đánh giá trong tập huấn luyện và tập kiểm tra
X_train_embeddings = torch.cat([compute_embeddings(sentence) for sentence in X_train], dim=0)
X_test_embeddings = torch.cat([compute_embeddings(sentence) for sentence in X_test], dim=0)

# # Tính toán embeddings cho tất cả các câu đánh giá trong tập huấn luyện và tập kiểm tra
# from tqdm import tqdm
# X_train_embeddings = []
# for sentence in tqdm(X_train[0:10]):
#     X_train_embeddings.append(compute_embeddings(sentence))

In [78]:
# Sử dụng embeddings làm đặc trưng cho mô hình Random Forest
# Tiếp tục với các bước còn lại như bạn đã thực hiện trước đó
# Xây dựng mô hình Random Forest
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train_embeddings, y_train)

# Đánh giá mô hình trên tập kiểm tra
y_pred = rf_classifier.predict(X_test_embeddings)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.97      0.88      1637
           1       0.71      0.36      0.48       448
           2       0.98      0.49      0.66        93
           3       0.63      0.17      0.27        99

    accuracy                           0.79      2277
   macro avg       0.78      0.50      0.57      2277
weighted avg       0.78      0.79      0.76      2277



In [79]:
# Sử dụng embeddings làm đặc trưng cho mô hình Random Forest
# Tiếp tục với các bước còn lại như bạn đã thực hiện trước đó
# Xây dựng mô hình Random Forest
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train_embeddings, y_train)

# Đánh giá mô hình trên tập kiểm tra
y_pred = rf_classifier.predict(X_test_embeddings)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.97      0.88      1637
           1       0.70      0.38      0.49       448
           2       0.96      0.46      0.62        93
           3       0.68      0.17      0.27        99

    accuracy                           0.80      2277
   macro avg       0.78      0.49      0.57      2277
weighted avg       0.79      0.80      0.77      2277



In [80]:
from sklearn.neighbors import KNeighborsClassifier as KNN

model = KNN()
model.fit(X_train_embeddings, y_train)

# Đánh giá mô hình trên tập kiểm tra
y_pred = model.predict(X_test_embeddings)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.95      0.89      1637
           1       0.67      0.46      0.54       448
           2       0.87      0.73      0.80        93
           3       0.71      0.22      0.34        99

    accuracy                           0.81      2277
   macro avg       0.77      0.59      0.64      2277
weighted avg       0.80      0.81      0.79      2277



In [81]:
# Sử dụng embeddings làm đặc trưng cho mô hình Random Forest
# Tiếp tục với các bước còn lại như bạn đã thực hiện trước đó
# Xây dựng mô hình Random Forest
rf_classifier = RandomForestClassifier(class_weight={0: 0.05,
                                                     1: 0.1,
                                                     2: 0.35,
                                                    3: 0.35})
rf_classifier.fit(X_train_embeddings, y_train)

# Đánh giá mô hình trên tập kiểm tra
y_pred = rf_classifier.predict(X_test_embeddings)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.97      0.87      1637
           1       0.75      0.27      0.40       448
           2       0.96      0.51      0.66        93
           3       0.66      0.19      0.30        99

    accuracy                           0.78      2277
   macro avg       0.79      0.49      0.56      2277
weighted avg       0.78      0.78      0.74      2277



In [82]:
# Sử dụng embeddings làm đặc trưng cho mô hình Random Forest
# Tiếp tục với các bước còn lại như bạn đã thực hiện trước đó
# Xây dựng mô hình Random Forest
rf_classifier = RandomForestClassifier(class_weight={0: 0.05,
                                                     1: 0.15,
                                                     2: 0.4,
                                                    3: 0.4})
rf_classifier.fit(X_train_embeddings, y_train)

# Đánh giá mô hình trên tập kiểm tra
y_pred = rf_classifier.predict(X_test_embeddings)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.97      0.87      1637
           1       0.74      0.25      0.37       448
           2       0.96      0.47      0.63        93
           3       0.63      0.19      0.29        99

    accuracy                           0.78      2277
   macro avg       0.78      0.47      0.54      2277
weighted avg       0.77      0.78      0.73      2277



In [83]:
# Sử dụng embeddings làm đặc trưng cho mô hình Random Forest
# Tiếp tục với các bước còn lại như bạn đã thực hiện trước đó
# Xây dựng mô hình Random Forest
rf_classifier = RandomForestClassifier(class_weight={0: 0.1,
                                                     1: 0.15,
                                                     2: 0.45,
                                                    3: 0.3})
rf_classifier.fit(X_train_embeddings, y_train)

# Đánh giá mô hình trên tập kiểm tra
y_pred = rf_classifier.predict(X_test_embeddings)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.98      0.88      1637
           1       0.77      0.32      0.45       448
           2       0.96      0.57      0.72        93
           3       0.63      0.19      0.29        99

    accuracy                           0.80      2277
   macro avg       0.79      0.51      0.59      2277
weighted avg       0.79      0.80      0.76      2277

